In [1]:
import diffuser.utils as utils
import ipdb
from trajdata import AgentBatch, AgentType, UnifiedDataset
from torch.utils.data import DataLoader
import os
from diffuser.utils.training_ped import Trainer
from ldm.models.diffusion.ddpm import Diffusion
import time
from ldm.models.diffusion.ddim import DDIMSampler
import torch
from trajdata_utils.interactive_vis import plot_agent_batch_interactive
import copy

pybullet build time: May 20 2022 19:44:17


In [2]:
#-----------------------------------------------------------------------------#
#---------------------------------- dataset ----------------------------------#
#-----------------------------------------------------------------------------#

dataset = UnifiedDataset(
    desired_data=["eupeds_eth", "eupeds_hotel", "eupeds_univ", "eupeds_zara1", "eupeds_zara2"],
    data_dirs={  # Remember to change this to match your filesystem!
        "eupeds_eth": "~/traj_dataset/eth-uty",
        "eupeds_hotel": "~/traj_dataset/eth-uty",
        "eupeds_univ": "~/traj_dataset/eth-uty",
        "eupeds_zara1": "~/traj_dataset/eth-uty",
        "eupeds_zara2": "~/traj_dataset/eth-uty",
    },
    history_sec =(0.0,  0.0), 
    future_sec =(15.9,  15.9),
    desired_dt=0.1
)

dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=dataset.get_collate_fn(),
    num_workers=0, # This can be set to 0 for single-threaded loading, if desired.
)


In [3]:
#-----------------------------------------------------------------------------#
#-------------------------------- instantiate --------------------------------#
#-----------------------------------------------------------------------------#

horizon = 160

diffusion = Diffusion(
    horizon = horizon
).cuda()

timestr = time.strftime("%Y%m%d-%H%M")
output_path = "./logs/eupeds/"+timestr
os.makedirs(output_path, exist_ok=True)

trainer = Trainer(
    diffusion_model=diffusion,
    data_loader=dataloader,
    results_folder=output_path
)

Diffusion: Running in x0-prediction mode
[ models/temporal ] Channel dimensions: [(4, 32), (32, 64), (64, 128), (128, 256)]
[(4, 32), (32, 64), (64, 128), (128, 256)]
UNetModel has 3.95 M params.


In [4]:
# trainer.load(0,'../logs/eupeds/20231114-1826/latest.pt')
trainer.load(0,'../logs/eupeds/diffuser/latest.pt')
sampler = DDIMSampler(trainer.model)

In [35]:
data = iter(dataloader)
batch = next(data)

In [36]:
pos = torch.cat((batch.agent_hist.position, batch.agent_fut.position), axis=1).transpose(1,2).cuda()
vel = torch.cat((batch.agent_hist.velocity, batch.agent_fut.velocity), axis=1).transpose(1,2).cuda()

pos = pos/10 - 0.5
vel = vel/10
traj = torch.cat((pos, vel), axis=1)

cond = pos[:,:2,(0,-1)]

In [48]:
ddim_steps = 500
ddim_eta = 0.0

traj_gen, _ = sampler.sample(S=ddim_steps,
                        conditioning=cond,
                        batch_size=1,
                        shape=[4, 160],
                        verbose=False,
                        eta=ddim_eta)

traj_gen[:,:2,:] = (traj_gen[:,:2,:]+0.5)*10
traj_gen[:,2:4,:] = traj_gen[:,2:4,:]*10

Data shape for DDIM sampling is (1, 4, 160), eta 0.0
Running DDIM Sampling with 500 timesteps


DDIM Sampler: 100%|████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 363.63it/s]


In [49]:
traj_gen = traj_gen.transpose(1,2)

In [50]:
plot_agent_batch_interactive(batch, batch_idx=0, cache_path=dataset.cache_path)

In [51]:
batch_gen = copy.deepcopy(batch)
batch_gen.agent_hist.position = traj_gen[:,0,:2].cpu()
batch_gen.agent_fut.position = traj_gen[:,1:,:2].cpu()
batch_gen.agent_hist.velocity = traj_gen[:,0,2:].cpu()
batch_gen.agent_fut.velocity = traj_gen[:,1:,2:].cpu()

In [52]:
plot_agent_batch_interactive(batch_gen, batch_idx=0, cache_path=dataset.cache_path)

In [42]:
traj_gen[0,-1,:2]

tensor([0.0228, 0.0000], device='cuda:0')

In [43]:
traj[0,-1,:2]

tensor([5.3178e-19, 4.8344e-19], device='cuda:0')

In [44]:
cond

tensor([[[-0.5000, -0.4977],
         [-0.5000, -0.5000]]], device='cuda:0')

In [45]:
(-0.4977+0.5)*10

0.023000000000000242